In [76]:
import pandas as pd
import numpy as np
import json
import pickle
import keras
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten

In [77]:
df = pd.read_csv('/content/drive/MyDrive/btp/sarcasm_with_id.csv')

embeddings = pickle.load(open('/content/drive/MyDrive/btp/sarcasm_aug_embeddings.pkl','rb'))

features = pickle.load(open('/content/drive/MyDrive/vit_features.pkl','rb'))


In [84]:
backtranslated = False

labels = []
combined = []
only_text = []
only_video = []

for index,row in df.iterrows():
  if(backtranslated==False and row['id'][-1]=='2'):
    continue
  a = embeddings[row['id']]
  b = np.mean(features[row['id'][:-2]],axis=0)
  c = np.concatenate((a,b))
  only_text.append(a)
  only_video.append(b)
  combined.append(c)
  labels.append(row['sarcasm'])

only_text = np.array(only_text)
only_video = np.array(only_video)
combined = np.array(combined)
labels = np.array(labels)

np.shape(combined)

(690, 1768)

In [79]:
from sklearn.utils import shuffle

def create_train_valid(features,labels,train_fraction = 0.7,max_valid=1000):

    features,labels = shuffle(features,labels,random_state = 50)

    train_end = max(int(train_fraction*len(labels)),len(labels)-max_valid)

    train_features = np.asarray(features[:train_end])
    valid_features = np.asarray(features[train_end:])

    train_labels = np.asarray(labels[:train_end])
    valid_labels = np.asarray(labels[train_end:])
    
    return train_features,valid_features,train_labels,valid_labels

In [88]:
inputs = only_video

x_train,x_valid,y_train,y_valid = create_train_valid(inputs,labels)

In [81]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler

def svm_train(features,labels):
    clf = make_pipeline(
        StandardScaler(),
        svm.SVC(C=15.0, gamma="scale", kernel="rbf")
    )
    return clf.fit(features, labels)

def svm_test(clf,features,labels):
    pred = clf.predict(features)
    true = labels

    result_string = classification_report(true, pred, digits=3)
    # print(confusion_matrix(true, pred))
    # print(result_string)
    return classification_report(true, pred, digits=3, output_dict=True)



In [91]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5,shuffle=True)
# skf.get_n_splits(inputs, labels)

results = []

for train_index, test_index in skf.split(inputs, labels):
  xtrain, xvalid = inputs[train_index], inputs[test_index]
  ytrain, yvalid = labels[train_index], labels[test_index]

  clf = svm_train(xtrain,ytrain)
  result_dict = svm_test(clf,xvalid,yvalid);
  results.append(result_dict)

acc = []

for re in results:
  acc.append(re["weighted avg"]["f1-score"])

np.mean(np.array(acc))

0.6901665411684387